# Monte Carlo Tree Search Lab

In this lab, we'll be using the game connect four, as a vehicle for learning MinMax and Monte Carlo Tree Search.
We'll also introduce concepts, such as state, that'll stay relevant throughout the course.
Expect to lose in connect four to the algorithm at the end of the lab.

## Setup
This section you won't need to edit, but it is worth skimming through—this is where we declare the objects you'll be interacting with througout the lab.

In [ ]:
# imports
import random
from typing import List, Tuple
import time
from copy import deepcopy # world -> thought

In [ ]:
# world and world model
class State:
    def __init__(self, cols=7, rows=6, win_req=4):
        self.board = [['.'] * cols for _ in range(rows)]
        self.heights = [1] * cols
        self.num_moves = 0
        self.win_req = win_req

    def get_avail_actions(self) -> List[int]:
        return [i for i in range(len(self.board[0])) if self.heights[i] <= len(self.board)]
  
    def put_action(self, action, agent):
        self.board[len(self.board) - self.heights[action]][action] = agent.name
        self.heights[action] += 1
        self.num_moves += 1

    def is_over(self):
        return self.num_moves >= len(self.board) * len(self.board[0])

    def __repr__(self):
        return self.__str__()
    
    def __str__(self):
        header  = " ".join([str(i) for i in range(len(self.board[0]))])
        line    = "".join(["-" for _ in range(len(header))])
        board   = [[e for e in row] for row in self.board]
        board   = '\n'.join([' '.join(row) for row in board])
        return  '\n' + header + '\n' + line + '\n' + board + '\n'


In [ ]:
# evaluate the utility of a state
def utility(state: 'State'):
    board = state.board
    n_cols = len(board[0]) - 1
    n_rows = len(board) - 1

    def diags_pos():
        """Get positive diagonals, going from bottom-left to top-right."""
        for di in ([(j, i - j) for j in range(n_cols)] for i in range(n_cols + n_rows - 1)):
            yield [board[i][j] for i, j in di if i >= 0 and j >= 0 and i < n_cols and j < n_rows]

    def diags_neg():
        """Get negative diagonals, going from top-left to bottom-right."""
        for di in ([(j, i - n_cols + j + 1) for j in range(n_cols)] for i in range(n_cols + n_rows - 1)):
            yield [board[i][j] for i, j in di if i >= 0 and j >= 0 and i < n_cols and j < n_rows]

    cols = list(map(list, list(zip(*board))))
    rows = board
    diags = list(diags_neg()) + list(diags_pos())
    lines = rows + cols + diags
    strings = ["".join(s) for s in lines]
    for string in strings:
        if 'OOOO' in string:
            return -1
        if 'XXXX' in string:
            return 1
    return 0


In [ ]:
# parrent class for mcts, minmax, human, and any other idea for an agent you have
class Agent:
    def __init__(self, name: str):
        self.name: str = name

    def get_action(self, state: State):
        return random.choice(state.get_avail_actions())


class Human:
    def __init__(self, name):
        self.name = name

    def get_action(self, state: State):
        given_input = int(input())
        if (given_input in state.get_avail_actions()):
            return given_input
        else:
            print('Pick a valid input')
        

In [ ]:
# connecting states and agents
class Game:
    def __init__(self, agents: Tuple[Gekko, Human]):
        self.agents = agents
        self.state = State()

    def play(self):
        while utility(self.state) == 0 and not self.state.is_over():
            for agent in agents:
                if utility(self.state) == 0 and not self.state.is_over():
                    action = agent.get_action(self.state)
                    self.state.put_action(action, agent)
                    print(self.state)

## Exercise 0: Discuss and Run game
Let's discuss if the `utility` function best belongs to the state or the agent.
Put the state, agent and game class together so that a game is run.

In [ ]:
agents = (Gekko('O'), Human('X'))
game = Game(agents)
game.play()

## Exercise 1: Human Agent
Make a child class of `Agent` called `Human`, with the `give_action` method overwritten to take input from you. *hint*: use `int(input())`

In [ ]:
class Human:
    def __init__(self, name):
        self.name = name


    def get_action(self, state: State):
        given_input = int(input())
        if (given_input in state.get_avail_actions()):
            return given_input 
        else:
            print ('Pick a valid input')
        


## Exercise 2: Gekko
Make a child class of `Agent` called `Gekko`, with a `get_action` that is very short sighted (greedy). You can basically do whatever you want here, as long as your output a valid action. You might want to make a `utility` function for the agent, and perhaps some helper functions. Write a two line comment explaining your Gekko's heuristic.

In [17]:
class Gekko:
    def __init__(self, name):
        self.name = name

    def determine_col_o_count(self, string):
        no_dots = string.replace(".", "")
        no_x_list = no_dots.split("X", 1)
        no_x_string = no_x_list[0]
        return len(no_x_string)

    def get_action(self, state: State):
        # Board stuff
        board = state.board
        col_height_list = state.heights
        n_cols = len(board[0]) - 1
        n_rows = len(board) - 1

        def diags_pos():
            """Get positive diagonals, going from bottom-left to top-right."""
            for di in ([(j, i - j) for j in range(n_cols)] for i in range(n_cols + n_rows - 1)):
                yield [board[i][j] for i, j in di if i >= 0 and j >= 0 and i < n_cols and j < n_rows]

        def diags_neg():
            """Get negative diagonals, going from top-left to bottom-right."""
            for di in ([(j, i - n_cols + j + 1) for j in range(n_cols)] for i in range(n_cols + n_rows - 1)):
                yield [board[i][j] for i, j in di if i >= 0 and j >= 0 and i < n_cols and j < n_rows]

        cols = list(map(list, list(zip(*board))))
        rows = board
        diags = list(diags_neg()) + list(diags_pos())
        lines = rows + cols + diags
        strings = ["".join(s) for s in lines]

        # New attempt at columns
        col_strings = ["".join(s) for s in cols]
        col_iterator = 0

        for col_height in col_height_list:
            # This is a valid column to pursue
            if col_strings[col_iterator][3] != "X" and col_strings[col_iterator][2] != "X" and col_strings[col_iterator][1] != "X" and col_strings[col_iterator][0] != "X" and col_height < 7:
                col_str = col_strings[col_iterator]
                o_count = self.determine_col_o_count(col_str)
                
                
                print("IF!")
            # This is NOT a valid column to pursue
            else: 
                print("ELSE!")
            
            col_iterator = col_iterator + 1


        # Greedy change, Only works for agent with name 'O':
        
        # # Doing the columns
        # col_strings = ["".join(s) for s in cols]
        # row_strings = ["".join(s) for s in rows]
        # # Can maybe do all in one loop
        # no_dots = 'TEST'
        # best_col_option = ''
        # best_col_option_place = 0
        # best_col_o_count = 0
        # current_col = 0
        # for string in col_strings:
        #     disallowed_characters = ".-1234567"
        #     for character in disallowed_characters:
        #         no_dots = string.replace(character, "")
        #     if best_col_option < no_dots:
        #         no_x_list = no_dots.split("X", 1)
        #         no_x_string = no_x_list[0]
        #         o_count = len(no_x_string)

        #         space_remaining = 6 - len(no_dots)
        #         space_needed = 4 - o_count
        #         if space_needed > space_remaining:
        #             # This is to try to avoid putting O's in a useless spot. Ideally, instead of setting the next best place to something semi random, 
        #             # having a 2nd best option to go back to would be ideal.
        #             if best_col_option_place == current_col:
        #                 if current_col == 0:
        #                     best_col_option_place = 1
        #                 else:
        #                     best_col_option_place = 6 - current_col
        #         else:
        #             if o_count > best_col_o_count:
        #                 best_col_o_count = o_count

        #             if o_count >= best_col_o_count and (len(no_dots) + space_needed) <= 6 :
        #                 best_col_option = no_dots
        #                 best_col_option_place = current_col
        #                 print("COL is: " + str(best_col_option_place))

        #             # Clear best_col_o_count if tied to bad option
        #             if best_col_option_place == current_col and (len(no_dots) + space_needed) <= 6:
        #                 best_col_o_count = 0
        #     current_col = current_col + 1
        
        # # Doing the rows (Which count from the top row, downwards)
        # best_row_option = ''
        # best_row_option_place = -1
        # best_row_o_count = 0
        # current_row = 0
        
        # for string in row_strings:
        #     o_count = 0
        #     is_possible_validator = 0
        #     valid_option_placements = []
        #     latest_x_place = 0
        #     current_char_place = 0
        #     for char in string:
        #         if char == "O":
        #             # increment o_count
        #             o_count = o_count + 1
        #             is_possible_validator = is_possible_validator + 1
        #             if o_count > best_row_o_count:
        #                 best_row_o_count = o_count
                
        #         if char == ".":
        #             is_possible_validator = is_possible_validator + 1
        #             # Here i should also check if building this up is better than pursuing something else
        #             # For now I just check if it is an option as it is
        #             if o_count > 0:
        #                 if current_row <= 4:
        #                     below_string = row_strings[current_row + 1]
        #                     bellow_char = below_string[current_char_place]
        #                     if bellow_char != ".":
        #                         valid_option_placements.append(current_char_place)
        #                         print("Testing if: " + str(valid_option_placements[len(
        #                             valid_option_placements) - 1]))
        #                 else:
        #                     valid_option_placements.append(current_char_place)
        #                     print("Testing else: " + str(valid_option_placements[len(
        #                         valid_option_placements) - 1]))

        #         if char == "X" or current_char_place >= 7:
        #             o_count = 0
        #             latest_x_place = current_char_place
        #             if is_possible_validator >= 4:
        #                 if len(valid_option_placements) > 0:
        #                     best_row_option_place = valid_option_placements[len(
        #                         valid_option_placements) - 1]
        #                     print("ROW is: " + str(best_row_option_place))
        #             else:
        #                 if best_row_option_place == current_row:
        #                     best_col_o_count = 0

        #         current_char_place = current_char_place + 1
        #     current_row = current_row + 1

        # if best_row_option_place >= 0 and best_row_o_count > best_col_o_count:
        #     return best_row_option_place
        # else:
        #     space_needed = 4 - best_col_o_count
        #     if best_col_option_place in state.get_avail_actions() and (len(best_col_option) + space_needed) <= 6:
        #         return best_col_option_place
        #     else:
        return random.choice(state.get_avail_actions())



# evaluate the utility of a state
def utility(state: 'State'):
    board = state.board
    n_cols = len(board[0]) - 1
    n_rows = len(board) - 1

    def diags_pos():
        """Get positive diagonals, going from bottom-left to top-right."""
        for di in ([(j, i - j) for j in range(n_cols)] for i in range(n_cols + n_rows - 1)):
            yield [board[i][j] for i, j in di if i >= 0 and j >= 0 and i < n_cols and j < n_rows]

    def diags_neg():
        """Get negative diagonals, going from top-left to bottom-right."""
        for di in ([(j, i - n_cols + j + 1) for j in range(n_cols)] for i in range(n_cols + n_rows - 1)):
            yield [board[i][j] for i, j in di if i >= 0 and j >= 0 and i < n_cols and j < n_rows]

    cols = list(map(list, list(zip(*board))))
    rows = board
    diags = list(diags_neg()) + list(diags_pos())
    lines = rows + cols + diags
    strings = ["".join(s) for s in lines]
            

    options = state.get_avail_actions()
    current_best = ''
    for string in strings:
        if 'OOOO' in string:
            return -1
        if 'XXXX' in string:
            return 1
    return 0


## *Optional exercise: MinMax*
Make a MinMax class:

In [ ]:
# class MinMax(Agent):
#   def __init__(self):
#       super(MinMax, self).__init__()

## Exercise 4: MCTS
Same but for Monte Carlo Tree Search. See if you can beat it.

In [ ]:
# class Node:
#     def __init__(self, state: State, parent: 'Node' = None):
#         self.children: List['Nodes'] = []
#         self.parent: 'Node' = parent
#         self.state: State = state

In [ ]:
# class MCTS(Agent):
#    def __init__(self):
#        super(MCTS, self).__init__()

## *Optional exercise: Dynamic Programming*
Then use dynamic programming to make your AI more efficient. You can use the class below (or not)

In [ ]:
class TranspositionTable:
    def __init__(self, size=1_000_000):
        self.size = size
        self.vals = [None] * size

    def board_str(self, state: State):
        return ''.join([''.join(c) for c in state.board])

    def put(self, state: State, utility: float):
        bstr = self.board_str(state)
        idx = hash(bstr) % self.size
        self.vals[idx] = (bstr, utility)

    def get(self, state: State):
        bstr = self.board_str(state)
        idx = hash(bstr) % self.size
        stored = self.vals[idx]
        if stored is None:
            return None
        if stored[0] == bstr:
            return stored[1]
        else:
            return None